**QC of ETL starting with GDC release 24 clinical tables**

This notebook focuses on the QC of program **CGCI** data_category clinical

This program has a total of five clinical tables present in this release

Tables listed below ---

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI`

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag`

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag_treat`

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow`

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow__mol_test`




##QC table checklist 

Multiple one-to-many tables present QC list

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

**2. Look at table row number and size**

Do these metrics make sense?

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

[ISB-CGC BigQuery table  search test tier](https://isb-cgc-test.appspot.com/bq_meta_search/)

Run a manual check in the console with the steps mentioned in step 1.

*Note from developer:
There are some columns which are sparsely populated (so they might look empty if you’re just scrolling through the table in the GUI), but there should be at least one non-null entry for every column in every table.*

**4. Number of case_id versus BigQuery metadata table**

**5.Check for any duplicate rows present in the table**

**7. Verify case_id count of table against  master rel_clinical_data table**

##Reference material



*   [NextGenETL](https://github.com/isb-cgc/NextGenETL) GitHub repository
*   [ETL QC SOP draft](https://docs.google.com/document/d/1Wskf3BxJLkMjhIXD62B6_TG9h5KRcSp8jSAGqcCP1lQ/edit)

##Before you begin

You need to load the BigQuery module, authenticate ourselves, create a client variable, and load the necessary libraries.


In [0]:
from google.colab import auth
try:
  auth.authenticate_user()
  print('You have been successfully authenticated!')
except:
  print('You have not been authenticated.')

You have been successfully authenticated!


In [0]:
from google.cloud import bigquery
try:
  project_id = 'isb-project-zero' # Update your_project_number with your project number
  client = bigquery.Client(project=project_id)
  print('BigQuery client successfully initialized')
except:
  print('Failed')

BigQuery client successfully initialized


In [0]:
#Install pypika to build a Query 
!pip install pypika
# Import from PyPika
from pypika import Query, Table, Field, Order

import pandas

     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for pypika: filename=PyPika-0.37.7-py2.py3-none-any.whl size=42747 sha256=945c90342fbaeaa181b11e0c39f69cd5020bbc698315ba5456e152ec49dbdc29
  Stored in directory: /root/.cache/pip/wheels/40/b2/20/cf67d3c67186b46241b5069c93da2c9beedbb3f08dba75fffe
Successfully built pypika


## READY TO BEGIN TESTING

##Clin CGCI 

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI`

[Table location](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel24_clin_CGCI&page=table)

Source : GDC API

Release version : v24


###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [0]:
#return all table information for rel24_clin_CGCI

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_CGCI,BASE TABLE


In [0]:
#return all table information for rel24_clin_CGCI

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [0]:
#check for empty schemas in dataset rel24_clin_CGCI

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below


In [0]:
#list of field descriptions for table rel24_clin_CGCI

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_CGCI

	submitter_id

	

rel24_clin_CGCI

	case_id

	

rel24_clin_CGCI

	diag__count

	Total child record count (located in cases table).

rel24_clin_CGCI

	follow__count

	Total child record count (located in cases table).

rel24_clin_CGCI

	primary_site

	

rel24_clin_CGCI

	disease_type

	

rel24_clin_CGCI

	index_date

	

rel24_clin_CGCI

	consent_type

	

rel24_clin_CGCI

	days_to_consent

	

rel24_clin_CGCI

	lost_to_followup

	

rel24_clin_CGCI

	proj__name

	Display name for the project

rel24_clin_CGCI

	proj__project_id

	

rel24_clin_CGCI

	demo__demographic_id

	

rel24_clin_CGCI

	demo__gender

	Text designations that identify gender. Gender is described as the assemblage of properties that distinguish people on the basis of their societal roles. [Explanatory Comment 1: Identification of gender is based upon self-report and may come from a form, questionnaire, interview, etc.]

rel24_clin_CGCI

	demo__race

	An arbitrary classification of a taxonomic group that i

In [0]:
# check for empty schemas in dataset rel24_clin_CGCI 

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
print(clin)

Are there any empty cells in the table schema?
         table_name  ...                                        description
0   rel24_clin_CGCI  ...                                                   
1   rel24_clin_CGCI  ...                                                   
2   rel24_clin_CGCI  ...  Total child record count (located in cases tab...
3   rel24_clin_CGCI  ...  Total child record count (located in cases tab...
4   rel24_clin_CGCI  ...                                                   
5   rel24_clin_CGCI  ...                                                   
6   rel24_clin_CGCI  ...                                                   
7   rel24_clin_CGCI  ...                                                   
8   rel24_clin_CGCI  ...                                                   
9   rel24_clin_CGCI  ...                                                   
10  rel24_clin_CGCI  ...                       Display name for the project
11  rel24_clin_CGCI  ...                 

###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(submitter_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI`

,f0_
0,332


In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_CGCI`

,f0_
0,120


In [0]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI`

,submitter_id,case_id,diag__count,follow__count,primary_site,disease_type,index_date,consent_type,days_to_consent,lost_to_followup,proj__name,proj__project_id,demo__demographic_id,demo__gender,demo__race,demo__ethnicity,demo__vital_status,demo__days_to_birth,demo__age_at_index,demo__days_to_death,demo__cause_of_death,demo__state,demo__created_datetime,demo__updated_datetime,expose__exposure_id,expose__tobacco_smoking_status,expose__tobacco_smoking_quit_year,expose__pack_years_smoked,expose__state,expose__created_datetime,expose__updated_datetime,state,created_datetime,updated_datetime
0,BLGSP-71-08-00508,1a81e2c4-b637-4028-ab28-4c64bc8bab76,1,0,Hematopoietic and reticuloendothelial systems,Mature B-Cell Lymphomas,None,None,NaN,None,Burkitt Lymphoma Genome Sequencing Project,CGCI-BLGSP,None,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN,NaN,None,None,None,released,2018-05-29T12:01:26.196634-05:00,2019-06-20T08:19:20.915742-05:00
1,BLGSP-71-08-00041,60acb146-43b3-44a3-9791-79284dae285e,1,2,Hematopoietic and reticuloendothelial systems,Mature B-Cell Lymphomas,Diagnosis,Informed Consent,-3.0,No,Burkitt Lymphoma Genome Sequencing Project,CGCI-BLGSP,35a37a1f-f724-5391-ac22-5879e46fce31,male,black or african american,not hispanic or latino,Dead,NaN,NaN,4.0,Cancer Related,released,2018-12-06T16:43:48.116079-06:00,2020-03-30T09:27:43.354105-05:00,None,NaN,NaN,NaN,None,None,None,released,2017-01-05T08:00:49.873737-06:00,2020-03-30T09:27:43.354105-05:00
2,BLGSP-71-08-00048,5fadbb10-de7f-482f-8460-54d513b5de89,1,2,Hematopoietic and reticuloendothelial systems,Mature B-Cell Lymphomas,Diagnosis,Informed Consent,-2.0,No,Burkitt Lymphoma Genome Sequencing Project,CGCI-BLGSP,7a1570fe-3391-5ce9-b067-5cb8ba7d0e1b,male,black or african american,not hispanic or latino,Dead,-1450.0,3.0,11.0,Not Cancer Related,released,2018-02-20T14:12:40.005144-06:00,2020-03-30T09:30:01.012029-05:00,None,NaN,NaN,NaN,None,None,None,released,2018-01-29T13:27:09.143885-06:00,2020-03-30T09:30:01.012029-05:00
3,BLGSP-71-08-00053,3adab9b3-da32-44df-9525-5f81e8cbfc02,1,2,Hematopoietic and reticuloendothelial systems,Mature B-Cell Lymphomas,Diagnosis,Informed Consent,-1.0,No,Burkitt Lymphoma Genome Sequencing Project,CGCI-BLGSP,8c996fda-2022-59d1-aa7d-0fd6466f8dfb,male,black or african american,not hispanic or latino,Dead,-3324.0,9.0,55.0,Cancer Related,released,2018-02-20T14:12:41.055987-06:00,2020-03-30T09:30:55.026350-05:00,None,NaN,NaN,NaN,None,None,None,released,2018-01-29T13:27:12.270900-06:00,2020-03-30T09:30:55.026350-05:00
4,BLGSP-71-08-00057,ef5d356c-89c9-48b1-a567-519aa478f166,1,2,Hematopoietic and reticuloendothelial systems,Mature B-Cell Lymphomas,Diagnosis,Informed Consent,-5.0,No,Burkitt Lymphoma Genome Sequencing Project,CGCI-BLGSP,45763d21-c7dc-5a0d-bd24-ba76c554f19b,male,black or african american,not hispanic or latino,Dead,-2325.0,6.0,9.0,Not Cancer Related,released,2018-05-30T12:35:37.020264-05:00,2020-03-30T09:31:30.800069-05:00,None,NaN,NaN,NaN,None,None,None,released,2018-01-29T13:27:13.467063-06:00,2020-03-30T09:31:30.800069-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,HTMCP-03-06-02333,18bce2c3-5060-4282-b7a0-11ada9a3eebf,1,10,Cervix uteri,Squamous Cell Neoplasms,Diagnosis,None,NaN,No,HIV+ Tumor Molecular Characterization Project ...,CGCI-HTMCP-CC,d41bc056-0c7f-5a6c-b5aa-4d46d6227ed7,female,black or african american,not hispanic or latino,Alive,-17396.0,47.0,NaN,None,released,2019-11-25T12:54:48.957558-06:00,2020-04-28T11:49:05.699379-05:00,51da84cf-8ab3-57fb-ae3c-d129126be989,1.0,NaN,NaN,released,2019-11-25T12:54:48.957558-06:00,2020-04-28T11:49:05.699379-05:00,released,2019-11-21T18:03:57.232011-06:00,2020-04-28T11:49:05.699379-05:00
328,HTMCP-03-06-02340,f07f2302-2276-4d5c-b52b-0b900464d39c,1,10,Cervix uteri,Squamous Cell Neoplasms,Diagnosis,None,NaN,No,HIV+ Tumor Molecular Characterization Project ...,CGCI-HTMCP-CC,468ade63

###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel24_clin_MMRF&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

*Note from developer:
There are some columns which are sparsely populated (so they might look empty if you’re just scrolling through the table in the GUI), but there should be at least one non-null entry for every column in every table.*

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**



In [0]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from submitter_id = ' + str(len(clin.index)))


number of case from submitter_id = 332


In [0]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'CGCI'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,8b9f18fb-9718-4012-8a61-9daf564cfba7,CGCI
1,1a6fbcfe-54d7-4785-9cf6-e596878d4101,CGCI
2,9220f346-6eab-4352-9b70-1889e3c957d8,CGCI
3,0c29ce4a-22ef-4b93-b9cc-b5a639168c2c,CGCI
4,31eee916-9abe-4cd8-a78a-423217affbc5,CGCI
...,...,...
327,8c9c36bd-ca35-4158-ae79-b317d3a61128,CGCI
328,d991215a-fd4f-434d-a17a-cf106aa72edf,CGCI
329,fcd39a3f-169a-463f-8153-17ebc1b218a8,CGCI
330,fcfdb847-5081-4086-8d32-25a281097390,CGCI


###match

In [0]:
%%bigquery --project isb-project-zero

SELECT distinct case_id, count(case_id) as count
FROM `isb-project-zero.GDC_metadata.rel24_fileData_current` as active, `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI` as clinical
WHERE program_name = 'CGCI'
AND active.case_gdc_id = clinical.case_id
group by case_id
order by count

,case_id,count
0,1a6fbcfe-54d7-4785-9cf6-e596878d4101,2
1,9220f346-6eab-4352-9b70-1889e3c957d8,2
2,0c29ce4a-22ef-4b93-b9cc-b5a639168c2c,2
3,31eee916-9abe-4cd8-a78a-423217affbc5,2
4,8b9f18fb-9718-4012-8a61-9daf564cfba7,3
...,...,...
327,0f4cddb3-c60e-4d95-b90a-3e26756a7bfc,17
328,90e9183f-7210-4922-a653-21c86e488887,17
329,0ceb3081-4c6f-4fd9-9535-5accddf76bc4,17
330,a7dfd3c9-4b94-4802-97c6-a096b276aeaf,17


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**


In [0]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI`
group by submitter_id, case_id, diag__count, follow__count, primary_site, disease_type, index_date, consent_type, days_to_consent, lost_to_followup, proj__name, proj__project_id, demo__demographic_id, demo__gender, demo__race, demo__ethnicity, demo__vital_status, demo__days_to_birth, demo__age_at_index, demo__days_to_death, demo__cause_of_death, demo__state, demo__created_datetime, demo__updated_datetime, expose__exposure_id, expose__tobacco_smoking_status, expose__tobacco_smoking_quit_year, expose__pack_years_smoked, expose__state, expose__created_datetime, expose__updated_datetime, state, created_datetime, updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [0]:
# case_id count from the program CGCI clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI` 
group by case_id
order by count

,case_id,count
0,1a81e2c4-b637-4028-ab28-4c64bc8bab76,1
1,60acb146-43b3-44a3-9791-79284dae285e,1
2,5fadbb10-de7f-482f-8460-54d513b5de89,1
3,3adab9b3-da32-44df-9525-5f81e8cbfc02,1
4,ef5d356c-89c9-48b1-a567-519aa478f166,1
...,...,...
327,18bce2c3-5060-4282-b7a0-11ada9a3eebf,1
328,f07f2302-2276-4d5c-b52b-0b900464d39c,1
329,9d05b62c-6948-4a15-8599-503436349fbc,1
330,89c3d38f-8eda-4f62-bb9d-8d3b818c969f,1


In [0]:
# case_id count from the master clinical table

%%bigquery --project isb-project-zero

SELECT distinct case_id, count(case_id) as count
FROM `isb-project-zero.GDC_metadata.rel24_fileData_current` as active, `isb-project-zero.GDC_Clinical_Data.rel24_clinical_data` as clinical
WHERE program_name = 'CGCI'
AND active.case_gdc_id = clinical.case_id
group by case_id
order by count


,case_id,count
0,1a6fbcfe-54d7-4785-9cf6-e596878d4101,2
1,9220f346-6eab-4352-9b70-1889e3c957d8,2
2,0c29ce4a-22ef-4b93-b9cc-b5a639168c2c,2
3,31eee916-9abe-4cd8-a78a-423217affbc5,2
4,8b9f18fb-9718-4012-8a61-9daf564cfba7,3
...,...,...
327,0f4cddb3-c60e-4d95-b90a-3e26756a7bfc,17
328,90e9183f-7210-4922-a653-21c86e488887,17
329,0ceb3081-4c6f-4fd9-9535-5accddf76bc4,17
330,a7dfd3c9-4b94-4802-97c6-a096b276aeaf,17


###match

##Clin CGCI_diag

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag`

[Table location](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel24_clin_CGCI_diag&page=table)

Source : GDC API

Release version : v24


###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [0]:
#return all table information for rel24_clin_CGCI_diag

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_CGCI_diag,BASE TABLE


In [0]:
#return all table information for rel24_clin_CGCI_diag

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [0]:
#check for empty schemas in dataset rel24_clin_CGCI_diag

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diagt') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below


In [0]:
#list of field descriptions for table rel24_clin_CGCI_diag

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_CGCI_diag

	diag__diagnosis_id

	Reference to ancestor diag__diagnosis_id, located in rel24_clin_CGCI_diag.

rel24_clin_CGCI_diag

	case_id

	Reference to ancestor case_id, located in rel24_clin_CGCI.

rel24_clin_CGCI_diag

	diag__treat__count

	Total child record count (located in cases.diagnoses table).

rel24_clin_CGCI_diag

	diag__ajcc_clinical_n

	Extent of the regional lymph node involvement for the cancer based on evidence obtained from clinical assessment parameters determined prior to treatment.

rel24_clin_CGCI_diag

	diag__ajcc_clinical_m

	Extent of the distant metastasis for the cancer based on evidence obtained from clinical assessment parameters determined prior to treatment.

rel24_clin_CGCI_diag

	diag__primary_diagnosis

	Text term used to describe the patient's histologic diagnosis, as described by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O).

rel24_clin_CGCI_diag

	diag__days_to_last_known_disease_st

In [0]:
# check for empty schemas in dataset rel24_clin_CGCI_diag

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
print(clin)

              table_name  ...                                        description
0   rel24_clin_CGCI_diag  ...  Reference to ancestor diag__diagnosis_id, loca...
1   rel24_clin_CGCI_diag  ...  Reference to ancestor case_id, located in rel2...
2   rel24_clin_CGCI_diag  ...  Total child record count (located in cases.dia...
3   rel24_clin_CGCI_diag  ...  Extent of the regional lymph node involvement ...
4   rel24_clin_CGCI_diag  ...  Extent of the distant metastasis for the cance...
5   rel24_clin_CGCI_diag  ...  Text term used to describe the patient's histo...
6   rel24_clin_CGCI_diag  ...  Time interval from the date of last follow up ...
7   rel24_clin_CGCI_diag  ...  Numeric value to represent the year of an indi...
8   rel24_clin_CGCI_diag  ...  The extent of a cervical or endometrial cancer...
9   rel24_clin_CGCI_diag  ...  Number of days between the date used for index...
10  rel24_clin_CGCI_diag  ...  Yes/No/Unknown indicator to identify whether a...
11  rel24_clin_CGCI_diag  ..

###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag`

,f0_
0,337


In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_CGCI_diag`

,f0_
0,123


In [0]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag`

,diag__diagnosis_id,case_id,diag__treat__count,diag__ajcc_clinical_n,diag__ajcc_clinical_m,diag__primary_diagnosis,diag__days_to_last_known_disease_status,diag__year_of_diagnosis,diag__figo_stage,diag__days_to_diagnosis,diag__progression_or_recurrence,diag__ajcc_pathologic_m,diag__site_of_resection_or_biopsy,diag__age_at_diagnosis,diag__ajcc_clinical_t,diag__days_to_last_follow_up,diag__lymph_nodes_tested,diag__tumor_grade,diag__tumor_largest_dimension_diameter,diag__last_known_disease_status,diag__pregnant_at_diagnosis,diag__ann_arbor_extranodal_involvement,diag__burkitt_lymphoma_clinical_variant,diag__synchronous_malignancy,diag__lymph_node_involved_site,diag__morphology,diag__ajcc_pathologic_t,diag__figo_staging_edition_year,diag__classification_of_tumor,diag__ann_arbor_pathologic_stage,diag__ann_arbor_b_symptoms,diag__tumor_stage,diag__prior_treatment,diag__ajcc_pathologic_n,diag__method_of_diagnosis,diag__tissue_or_organ_of_origin,diag__prior_malignancy,diag__state,diag__created_datetime,diag__updated_datetime
0,3b5d0e6e-0d0a-5d5c-acab-191b01340189,3cfb178a-c476-4c0f-968e-55c57239e7c7,3,None,None,Burkitt-like lymphoma,NaN,2015.0,None,NaN,no,None,Not Reported,980.0,None,NaN,NaN,Not Reported,NaN,With tumor,None,None,None,No,None,9680/3,None,NaN,None,None,None,None,No,None,Incisional Biopsy,"Hematopoietic system, NOS",no,released,2017-07-19T13:45:38.011076-05:00,2020-03-27T13:03:19.871592-05:00
1,bb6327d9-48ce-5288-8ff9-7bf0b3c14d2a,56105a23-4ced-4511-aa90-b956db97b881,7,None,None,Burkitt-like lymphoma,204.0,2013.0,None,NaN,no,None,"Mouth, NOS",NaN,None,1.0,NaN,Not Reported,11.0,With tumor,None,Yes,Endemic,No,None,9680/3,None,NaN,None,Stage III,Yes,None,No,None,Incisional Biopsy,"Hematopoietic system, NOS",no,released,2016-12-15T10:00:37.003006-06:00,2020-03-28T11:45:31.860689-05:00
2,74b1deb4-5108-5d06-b738-755aab6603d0,886e1609-e89c-4d5e-ab9d-4f9ae18c937a,3,None,None,Burkitt-like lymphoma,1.0,2014.0,None,NaN,no,None,"Abdomen, NOS",2475.0,None,NaN,NaN,Not Reported,15.0,With tumor,None,Yes,Endemic,No,None,9680/3,None,NaN,None,Stage III,Yes,None,No,None,Biopsy,"Hematopoietic system, NOS",no,released,2018-05-30T12:35:04.178051-05:00,2020-03-27T12:28:47.289163-05:00
3,0e3c31ff-f17e-5a68-9398-936fbee8f391,0024c94c-88ff-49d9-8dc4-bf77f832d85e,5,None,None,Burkitt-like lymphoma,57.0,2014.0,None,NaN,no,None,Not Reported,4943.0,None,NaN,NaN,Not Reported,2.0,With tumor,None,Yes,None,No,Cervical,9680/3,None,NaN,None,Stage III,Yes,None,No,None,Incisional Biopsy,"Hematopoietic system, NOS",no,released,2017-04-25T11:28:04.260680-05:00,2020-03-28T12:21:49.045488-05:00
4,229208c6-7fe9-5a11-b3b8-6e78a7389d5a,2a587474-c6c3-4322-b1ad-a11f2d205bb4,3,None,None,Burkitt-like lymphoma,2.0,2014.0,None,NaN,no,None,Intra-abdominal lymph nodes,NaN,None,1.0,NaN,Not Reported,10.0,With tumor,None,Yes,Endemic,No,Retroperitoneal,9680/3,None,NaN,None,Stage IV,Yes,None,No,None,Biopsy,"Hematopoietic system, NOS",no,released,2018-02-20T14:12:23.007536-06:00,2020-03-27T12:35:53.914487-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,17fcdc5d-6e18-549e-ae40-305d56ad6357,e809c820-4623-4a88-8718-ea402b041a41,6,NX,MX,"Adenocarcinoma, NOS",319.0,2014.0,Stage IIIB,NaN,no,None,Cervix uteri,NaN,T3b,NaN,NaN,G3,NaN,With tumor,No,None,None,No,None,8140/3,None,2009.0,None,None,None,None,No,None,Biopsy,Cervix uteri,no,released,2019-11-25T12:53:54.756047-06:00,2020-04-28T11:49:05.699379-05:00
333,785c8ce0-092e-5dec-80d6-dea36957e04c,1eb3e434-a4b6-4ca3-868e-ff6b6a4d1dab,3,N0,M0,"Adenocarcinoma, NOS",91.0,2016.0,Stage IIIB,NaN,no,None,Cervix uteri,NaN,T3b,NaN,NaN,G3,NaN,With tumor,No,None,None,No,None,8140/3,None,2009.0,None,None,None,None,No,None,Biopsy,Cervix uteri,no,released,2019-11-25T12:54:26.380820-06:00,2020-04-28T11:49:05.699379-05:00
334,574c32d2-8342-5e2c-9dde-f9a04264c18c,1105dd98-0a06-4938-97c2-7e1582944e8d,5,N0,M0,"Adenocarcinoma, NOS",71.0,

###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel24_clin_MMRF_diag__treat&page=table).

Run a manual check in the console with the steps mentioned in step 1.

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**



In [0]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from submitter_id = ' + str(len(clin.index)))


number of case from submitter_id = 332


###match

In [0]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'CGCI'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,8b9f18fb-9718-4012-8a61-9daf564cfba7,CGCI
1,1a6fbcfe-54d7-4785-9cf6-e596878d4101,CGCI
2,9220f346-6eab-4352-9b70-1889e3c957d8,CGCI
3,0c29ce4a-22ef-4b93-b9cc-b5a639168c2c,CGCI
4,31eee916-9abe-4cd8-a78a-423217affbc5,CGCI
...,...,...
327,8c9c36bd-ca35-4158-ae79-b317d3a61128,CGCI
328,d991215a-fd4f-434d-a17a-cf106aa72edf,CGCI
329,fcd39a3f-169a-463f-8153-17ebc1b218a8,CGCI
330,fcfdb847-5081-4086-8d32-25a281097390,CGCI


In [0]:
%%bigquery --project isb-project-zero

SELECT distinct case_id, count(case_id) as count
FROM `isb-project-zero.GDC_metadata.rel24_caseData` as active, `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag` as clinical
WHERE program_name = 'CGCI'
AND active.case_gdc_id = clinical.case_id
group by case_id
order by count

,case_id,count
0,8b9f18fb-9718-4012-8a61-9daf564cfba7,1
1,1a6fbcfe-54d7-4785-9cf6-e596878d4101,1
2,9220f346-6eab-4352-9b70-1889e3c957d8,1
3,0c29ce4a-22ef-4b93-b9cc-b5a639168c2c,1
4,31eee916-9abe-4cd8-a78a-423217affbc5,1
...,...,...
327,c5e55e09-1a50-4415-95e8-0db66771a2db,2
328,a21b8ec0-1c09-48cb-97d1-42ed0096964a,2
329,39dce88d-112c-4a3d-b2d2-11e0616594d8,2
330,d04504c8-766f-4829-9d11-cc32f3052f63,2


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**

In [0]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag`
group by diag__diagnosis_id, case_id, diag__treat__count, diag__ajcc_clinical_n, diag__ajcc_clinical_m, diag__primary_diagnosis, diag__days_to_last_known_disease_status, diag__year_of_diagnosis, diag__figo_stage, diag__days_to_diagnosis, diag__progression_or_recurrence, diag__ajcc_pathologic_m, diag__site_of_resection_or_biopsy, diag__age_at_diagnosis, diag__ajcc_clinical_t, diag__days_to_last_follow_up, diag__lymph_nodes_tested, diag__tumor_grade, diag__tumor_largest_dimension_diameter, diag__last_known_disease_status, diag__pregnant_at_diagnosis, diag__ann_arbor_extranodal_involvement, diag__burkitt_lymphoma_clinical_variant, diag__synchronous_malignancy, diag__lymph_node_involved_site, diag__morphology, diag__ajcc_pathologic_t, diag__figo_staging_edition_year, diag__classification_of_tumor, diag__ann_arbor_pathologic_stage, diag__ann_arbor_b_symptoms, diag__tumor_stage, diag__prior_treatment, diag__ajcc_pathologic_n, diag__method_of_diagnosis, diag__tissue_or_organ_of_origin, diag__prior_malignancy, diag__state, diag__created_datetime, diag__updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [0]:
# case_id count from the program CGCI clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI` 
group by case_id
order by count

,case_id,count
0,1a81e2c4-b637-4028-ab28-4c64bc8bab76,1
1,60acb146-43b3-44a3-9791-79284dae285e,1
2,5fadbb10-de7f-482f-8460-54d513b5de89,1
3,3adab9b3-da32-44df-9525-5f81e8cbfc02,1
4,ef5d356c-89c9-48b1-a567-519aa478f166,1
...,...,...
327,18bce2c3-5060-4282-b7a0-11ada9a3eebf,1
328,f07f2302-2276-4d5c-b52b-0b900464d39c,1
329,9d05b62c-6948-4a15-8599-503436349fbc,1
330,89c3d38f-8eda-4f62-bb9d-8d3b818c969f,1


In [0]:
# case_id count from the program CGCI_diag clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag` 
group by case_id
order by count

,case_id,count
0,3cfb178a-c476-4c0f-968e-55c57239e7c7,1
1,56105a23-4ced-4511-aa90-b956db97b881,1
2,886e1609-e89c-4d5e-ab9d-4f9ae18c937a,1
3,0024c94c-88ff-49d9-8dc4-bf77f832d85e,1
4,2a587474-c6c3-4322-b1ad-a11f2d205bb4,1
...,...,...
327,a21b8ec0-1c09-48cb-97d1-42ed0096964a,2
328,39dce88d-112c-4a3d-b2d2-11e0616594d8,2
329,d04504c8-766f-4829-9d11-cc32f3052f63,2
330,c5e55e09-1a50-4415-95e8-0db66771a2db,2


###match

##Clin CGCI_diag__treat

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag__treat`

[Table location][Table location](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel24_clin_CGCI_diag__treat&page=table)

Source : GDC API

Release version : v24

###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [0]:
#return all table information for rel24_clin_CGCI_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag__treat') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_CGCI_diag__treat,BASE TABLE


In [0]:
#return all table information for rel24_clin_CGCI_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag__treatt') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [0]:
#check for empty schemas in dataset rel24_clin_CGCI_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag__treat') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below

In [0]:
#list of field descriptions for table rel24_clin_CGCI_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag__treat') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_CGCI_diag__treat

	diag__treat__treatment_id

	

rel24_clin_CGCI_diag__treat

	diag__diagnosis_id

	Reference to ancestor diag__diagnosis_id, located in rel24_clin_CGCI_diag.

rel24_clin_CGCI_diag__treat

	case_id

	Reference to ancestor case_id, located in rel24_clin_CGCI.

rel24_clin_CGCI_diag__treat

	diag__treat__days_to_treatment_start

	Number of days between the date used for index and the date the treatment started.

rel24_clin_CGCI_diag__treat

	diag__treat__number_of_cycles

	The numeric value used to describe the number of cycles of a specific treatment or regimen the patient received.

rel24_clin_CGCI_diag__treat

	diag__treat__treatment_outcome

	Text term that describes the patient's final outcome after the treatment was administered.

rel24_clin_CGCI_diag__treat

	diag__treat__chemo_concurrent_to_radiation

	The text term used to describe whether the patient was receiving chemotherapy concurrent to radiation.

rel24_clin_CGCI_diag__treat

	diag__treat__treatme

In [0]:
# check for empty schemas in dataset rel24_clin_CGCI_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_diag__treat') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
print(clin)

                     table_name  ...                                        description
0   rel24_clin_CGCI_diag__treat  ...                                                   
1   rel24_clin_CGCI_diag__treat  ...  Reference to ancestor diag__diagnosis_id, loca...
2   rel24_clin_CGCI_diag__treat  ...  Reference to ancestor case_id, located in rel2...
3   rel24_clin_CGCI_diag__treat  ...  Number of days between the date used for index...
4   rel24_clin_CGCI_diag__treat  ...  The numeric value used to describe the number ...
5   rel24_clin_CGCI_diag__treat  ...  Text term that describes the patient's final o...
6   rel24_clin_CGCI_diag__treat  ...  The text term used to describe whether the pat...
7   rel24_clin_CGCI_diag__treat  ...  Text term that describes the kind of treatment...
8   rel24_clin_CGCI_diag__treat  ...  The anatomic site or field targeted by a treat...
9   rel24_clin_CGCI_diag__treat  ...  A yes/no/unknown/not applicable indicator rela...
10  rel24_clin_CGCI_diag__treat 

###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag__treat`

,f0_
0,1897


In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_CGCI_diag__treat`

,f0_
0,519


In [0]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag__treat`

,diag__treat__treatment_id,diag__diagnosis_id,case_id,diag__treat__days_to_treatment_start,diag__treat__number_of_cycles,diag__treat__treatment_outcome,diag__treat__chemo_concurrent_to_radiation,diag__treat__treatment_type,diag__treat__treatment_anatomic_site,diag__treat__treatment_or_therapy,diag__treat__treatment_dose,diag__treat__therapeutic_agents,diag__treat__initial_disease_status,diag__treat__days_to_treatment_end,diag__treat__treatment_frequency,diag__treat__regimen_or_line_of_therapy,diag__treat__treatment_intent_type,diag__treat__state,diag__treat__created_datetime,diag__treat__updated_datetime
0,319f62c1-48bf-47ec-97e3-014649aa06c4,167a9d75-746e-52f4-b8b2-3b9dc6184beb,58576f61-cf0d-4542-9f88-257ee412af7f,122.0,NaN,None,None,"Radiation Therapy, NOS",Brain,yes,20.0,None,None,131.0,None,None,Palliative,released,2020-03-28T12:21:18.335483-05:00,2020-04-02T17:04:15.842910-05:00
1,3e18210d-cb0e-4984-b5a9-a599c99b1dfe,26a6fded-d799-5f98-a743-f81ca60cbeea,a7547cd6-ffc3-4ea3-a123-c80c75ff85f5,205.0,NaN,Stable Disease,None,"Radiation Therapy, NOS","Head - Face Or Neck, Nos",yes,49.0,None,None,239.0,None,None,Adjuvant,released,2020-03-28T12:24:35.215483-05:00,2020-04-02T17:04:15.842910-05:00
2,f2b96644-d474-4041-91d5-d7fcf8014ea9,26a6fded-d799-5f98-a743-f81ca60cbeea,a7547cd6-ffc3-4ea3-a123-c80c75ff85f5,223.0,NaN,None,None,"Radiation Therapy, NOS",None,yes,99.0,None,Initial Diagnosis,245.0,None,None,None,released,2020-03-28T12:24:35.215483-05:00,2020-04-02T17:04:15.842910-05:00
3,8a455716-dc98-4ac2-9032-36f1e12234fc,56405ec3-cfe2-523c-b6ae-19c7c871576a,2ef3e472-4a43-48a5-917f-c336194c0373,239.0,NaN,Progressive Disease,None,"Radiation Therapy, NOS",None,yes,99.0,None,Progressive Disease,242.0,None,None,None,released,2020-03-28T11:45:58.652463-05:00,2020-04-02T17:04:15.842910-05:00
4,8500af05-b716-4dab-8f3f-8f0184df3381,119ad3be-9c5c-507b-8888-fb114d325a4b,80d31738-d3f2-42e6-871a-f0612812a678,211.0,NaN,None,None,"Radiation Therapy, NOS",None,yes,15.0,None,Initial Diagnosis,233.0,None,None,None,released,2020-03-28T10:00:58.752404-05:00,2020-04-28T11:49:05.699379-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,cc51f7ae-3c0e-43d6-b167-e3234c1f3044,22b547f8-1985-519e-978e-6b790d0e1bfb,9e3c05ba-ba46-4e96-bdff-660ca34d6a4c,NaN,NaN,Stable Disease,None,"Brachytherapy, Low Dose",Cervix,yes,3000.0,None,None,NaN,None,None,Adjuvant,released,2020-03-28T10:32:19.820817-05:00,2020-04-28T11:49:05.699379-05:00
1893,c97d29b3-61f5-4ba2-9e9c-b0369ad7f014,7c239180-af45-5efc-b332-5179480c9990,f429c531-358b-413d-ba3e-50ea945e68cf,NaN,NaN,Complete Response,None,"Brachytherapy, Low Dose",Cervix,yes,3000.0,None,None,NaN,None,None,Adjuvant,released,2020-03-28T10:32:42.402497-05:00,2020-04-28T11:49:05.699379-05:00
1894,f51a8a12-e7d2-4a43-85cb-fbbb216f91eb,e8893be9-31fe-5e5a-906e-1c10381e9428,796408b3-5f54-4ef5-afeb-b224fc0eb3e6,NaN,NaN,Complete Response,None,"Brachytherapy, Low Dose",Cervix,yes,3000.0,None,None,NaN,None,None,Adjuvant,released,2020-03-28T10:55:20.689165-05:00,2020-04-28T11:49:05.699379-05:00
1895,877cedc5-eb46-43bb-9d45-9cd8d33395b8,3277b040-7136-546a-9ee3-0d5619a83bce,408b076a-0eee-4bc4-84eb-f5590c6f2a0f,NaN,NaN,None,None,"Brachytherapy, Low Dose",Cervix,yes,3000.0,None,None,NaN,None,None,Adjuvant,released,2020-03-28T10:24:48.302389-05:00,2020-04-28T11:49:05.699379-05:00


###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_clin_CGCI_diag__treat&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**

In [0]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag__treat`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from case_id = ' + str(len(clin.index)))

number of case from case_id = 331


In [0]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'CGCI'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,8b9f18fb-9718-4012-8a61-9daf564cfba7,CGCI
1,1a6fbcfe-54d7-4785-9cf6-e596878d4101,CGCI
2,9220f346-6eab-4352-9b70-1889e3c957d8,CGCI
3,0c29ce4a-22ef-4b93-b9cc-b5a639168c2c,CGCI
4,31eee916-9abe-4cd8-a78a-423217affbc5,CGCI
...,...,...
327,8c9c36bd-ca35-4158-ae79-b317d3a61128,CGCI
328,d991215a-fd4f-434d-a17a-cf106aa72edf,CGCI
329,fcd39a3f-169a-463f-8153-17ebc1b218a8,CGCI
330,fcfdb847-5081-4086-8d32-25a281097390,CGCI


### no match 

Explanation: Row count doesn't match due to not all cases contain certain field groups. 

In [0]:
%%bigquery --project isb-project-zero

SELECT distinct case_id, count(case_id) as count
FROM `isb-project-zero.GDC_metadata.rel24_caseData` as active, `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag__treat` as clinical
WHERE program_name = 'CGCI'
AND active.case_gdc_id = clinical.case_id
group by case_id
order by count

,case_id,count
0,3adab9b3-da32-44df-9525-5f81e8cbfc02,1
1,31c738af-68e4-4890-9671-4028c21cbf26,1
2,6e6440c2-750d-4440-a9ee-8f0f5c017093,1
3,00bd58bd-223d-433e-b60a-5bf355f342b1,1
4,b5b83242-ef5d-4922-815b-05f88000dcd6,1
...,...,...
326,a6b6d55e-e627-4a60-a95a-72db12bb25dc,14
327,c2282c90-795d-4c90-9c4b-51e340f98ac4,14
328,796408b3-5f54-4ef5-afeb-b224fc0eb3e6,16
329,2ef3e472-4a43-48a5-917f-c336194c0373,16


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**

In [0]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag__treat`
group by diag__treat__treatment_id, diag__diagnosis_id, case_id, diag__treat__days_to_treatment_start, diag__treat__number_of_cycles, diag__treat__treatment_outcome, diag__treat__chemo_concurrent_to_radiation, diag__treat__treatment_type, diag__treat__treatment_anatomic_site, diag__treat__treatment_or_therapy, diag__treat__treatment_dose, diag__treat__therapeutic_agents, diag__treat__initial_disease_status, diag__treat__days_to_treatment_end, diag__treat__treatment_frequency, diag__treat__regimen_or_line_of_therapy, diag__treat__treatment_intent_type, diag__treat__state, diag__treat__created_datetime, diag__treat__updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [0]:
# case_id count from the program CGCI clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI` 
group by case_id
order by count

,case_id,count
0,1a81e2c4-b637-4028-ab28-4c64bc8bab76,1
1,60acb146-43b3-44a3-9791-79284dae285e,1
2,5fadbb10-de7f-482f-8460-54d513b5de89,1
3,3adab9b3-da32-44df-9525-5f81e8cbfc02,1
4,ef5d356c-89c9-48b1-a567-519aa478f166,1
...,...,...
327,18bce2c3-5060-4282-b7a0-11ada9a3eebf,1
328,f07f2302-2276-4d5c-b52b-0b900464d39c,1
329,9d05b62c-6948-4a15-8599-503436349fbc,1
330,89c3d38f-8eda-4f62-bb9d-8d3b818c969f,1


In [0]:
# case_id count from the program CGCI_diag__treat clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag__treat` 
group by case_id
order by count

,case_id,count
0,39ded2c0-56b5-4561-aa3f-7d417a32a131,1
1,89335258-7991-4e19-87e2-60bf747d89bd,1
2,be578313-e34c-4f26-aba6-4dc05dfd6f4d,1
3,f90e6333-f59e-4f6b-b7f9-17ba10e4a600,1
4,48823727-d7d5-4efa-b374-17e3cfffad91,1
...,...,...
326,a6b6d55e-e627-4a60-a95a-72db12bb25dc,14
327,c2282c90-795d-4c90-9c4b-51e340f98ac4,14
328,2ef3e472-4a43-48a5-917f-c336194c0373,16
329,796408b3-5f54-4ef5-afeb-b224fc0eb3e6,16


### no match 

Explanation: Row count doesn't match due to not all cases contain certain field groups. 

###test 7 - disgnosis_id count verification

**7. QC diagnosis_id count from parent diag table if applicable**

In [0]:
# diag_diagnosis_id count from the program CGCI_diag clinical table

%%bigquery --project isb-project-zero

select distinct diag__diagnosis_id, count(diag__diagnosis_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag` 
group by diag__diagnosis_id
order by count

,diag__diagnosis_id,count
0,3b5d0e6e-0d0a-5d5c-acab-191b01340189,1
1,bb6327d9-48ce-5288-8ff9-7bf0b3c14d2a,1
2,74b1deb4-5108-5d06-b738-755aab6603d0,1
3,0e3c31ff-f17e-5a68-9398-936fbee8f391,1
4,229208c6-7fe9-5a11-b3b8-6e78a7389d5a,1
...,...,...
332,17fcdc5d-6e18-549e-ae40-305d56ad6357,1
333,785c8ce0-092e-5dec-80d6-dea36957e04c,1
334,574c32d2-8342-5e2c-9dde-f9a04264c18c,1
335,be52ff37-7347-5865-bd4a-e2a7a3bc883f,1


In [0]:
# diag_diagnosis_id count from the program CGCI_diag__treat clinical table

%%bigquery --project isb-project-zero

select distinct diag__diagnosis_id, count(diag__diagnosis_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_diag__treat` 
group by diag__diagnosis_id
order by count

,diag__diagnosis_id,count
0,f8ade3a0-a327-50c2-abe5-94247bd0c9a1,1
1,9ef47e2b-6613-53bd-95e9-70ab73fdbe33,1
2,7b03b5bc-1aa0-5f89-a914-9b960180289e,1
3,b9264437-6559-5aa9-9591-636833ca3161,1
4,7432c344-1351-5011-8289-8147e4f3d047,1
...,...,...
330,8aff4393-c8a0-52ad-b70f-fbd23038e520,14
331,3d537c9e-8e38-53fb-9cba-df0b7eafc1e5,14
332,56405ec3-cfe2-523c-b6ae-19c7c871576a,16
333,e8893be9-31fe-5e5a-906e-1c10381e9428,16


### no match 

Explanation: parent table count returns 337 rows, while the treat table has 335 rows present

##Clin CGCI_follow

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel23_clin_CGCI_follow`

[Table location](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_clin_CGCI_follow&page=table)

Source : GDC API

Release version : v24

###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [0]:
#return all table information for rel24_clin_CGCI_follow

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_CGCI_follow,BASE TABLE


In [0]:
#return all table information for rel24_clin_CGCI_follow

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [0]:
#check for empty schemas in dataset rel24_clin_CGCI_follow

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below




In [0]:
#list of field descriptions for table rel24_clin_CGCI_follow

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_CGCI_follow

	follow__follow_up_id

	Reference to ancestor follow__follow_up_id, located in rel24_clin_CGCI_follow.

rel24_clin_CGCI_follow

	case_id

	Reference to ancestor case_id, located in rel24_clin_CGCI.

rel24_clin_CGCI_follow

	follow__mol_test__count

	Total child record count (located in cases.follow_ups table).

rel24_clin_CGCI_follow

	follow__days_to_follow_up

	Number of days between the date used for index and the date of the patient's last follow-up appointment or contact.

rel24_clin_CGCI_follow

	follow__height

	The height of the patient in centimeters.

rel24_clin_CGCI_follow

	follow__weight

	The weight of the patient measured in kilograms.

rel24_clin_CGCI_follow

	follow__bmi

	A calculated numerical quantity that represents an individual's weight to height ratio.

rel24_clin_CGCI_follow

	follow__evidence_of_recurrence_type

	The text term used to describe the type of evidence used to determine whether the patient's disease recurred.

rel24_clin_CGC

In [0]:
# check for empty schemas in dataset rel24_clin_CGCI_follow
clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
print(clin)

                table_name  ...                                        description
0   rel24_clin_CGCI_follow  ...  Reference to ancestor follow__follow_up_id, lo...
1   rel24_clin_CGCI_follow  ...  Reference to ancestor case_id, located in rel2...
2   rel24_clin_CGCI_follow  ...  Total child record count (located in cases.fol...
3   rel24_clin_CGCI_follow  ...  Number of days between the date used for index...
4   rel24_clin_CGCI_follow  ...          The height of the patient in centimeters.
5   rel24_clin_CGCI_follow  ...   The weight of the patient measured in kilograms.
6   rel24_clin_CGCI_follow  ...  A calculated numerical quantity that represent...
7   rel24_clin_CGCI_follow  ...  The text term used to describe the type of evi...
8   rel24_clin_CGCI_follow  ...  Number of days between the date used for index...
9   rel24_clin_CGCI_follow  ...  The text term used to describe a comorbidity d...
10  rel24_clin_CGCI_follow  ...  The text term used to describe the type of hys...
11  

###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow`

,f0_
0,9049


In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_CGCI_follow`

,f0_
0,9049


In [0]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow`

,follow__follow_up_id,case_id,follow__mol_test__count,follow__days_to_follow_up,follow__height,follow__weight,follow__bmi,follow__evidence_of_recurrence_type,follow__days_to_progression,follow__comorbidity,follow__hysterectomy_type,follow__menopause_status,follow__hormonal_contraceptive_use,follow__hiv_viral_load,follow__aids_risk_factors,follow__karnofsky_performance_status,follow__disease_response,follow__pregnancy_outcome,follow__cdc_hiv_risk_factors,follow__ecog_performance_status,follow__cd4_count,follow__progression_or_recurrence,follow__progression_or_recurrence_anatomic_site,follow__risk_factor,follow__haart_treatment_indicator,follow__days_to_recurrence,follow__nadir_cd4_count,follow__state,follow__created_datetime,follow__updated_datetime
0,27640af3-cdfb-4904-9341-d65f1b47517e,a21b8ec0-1c09-48cb-97d1-42ed0096964a,0,NaN,116.0,21.0,15.6,None,NaN,None,None,None,None,NaN,None,40.0,None,None,None,3,NaN,None,None,None,None,NaN,NaN,released,2020-03-28T14:12:32.147298-05:00,2020-04-02T17:04:15.842910-05:00
1,d175ae71-bbad-4e45-9d3c-771a3e9f02fb,3cfb178a-c476-4c0f-968e-55c57239e7c7,8,NaN,103.0,16.0,15.1,None,NaN,None,None,None,None,NaN,None,70.0,None,None,None,2,NaN,None,None,None,None,NaN,NaN,released,2020-03-27T13:03:19.871592-05:00,2020-04-02T17:04:15.842910-05:00
2,085e05df-f73c-46fb-a297-43f245e40491,39ded2c0-56b5-4561-aa3f-7d417a32a131,0,NaN,115.0,21.0,15.9,None,NaN,None,None,None,None,NaN,None,90.0,None,None,None,1,NaN,None,None,None,None,NaN,NaN,released,2020-03-30T09:18:00.864578-05:00,2020-04-02T17:04:15.842910-05:00
3,53de1b44-46b8-4656-90b2-ad41a6ff8e24,89335258-7991-4e19-87e2-60bf747d89bd,0,NaN,111.0,19.1,15.5,None,NaN,None,None,None,None,NaN,None,80.0,None,None,None,2,NaN,None,None,None,None,NaN,NaN,released,2020-03-30T09:21:04.649158-05:00,2020-04-02T17:04:15.842910-05:00
4,5b5fd590-d03c-481a-a7d4-3faed33bc3f1,be578313-e34c-4f26-aba6-4dc05dfd6f4d,0,NaN,NaN,13.0,NaN,None,NaN,None,None,None,None,NaN,None,30.0,None,None,None,4,NaN,None,None,None,None,NaN,NaN,released,2020-03-30T09:21:37.528786-05:00,2020-04-02T17:04:15.842910-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,9a24df76-6ce4-40a5-9c2f-0ab5da97930f,ab45050e-2965-4d33-bff4-cd7a067ddd3d,0,622.0,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,NaN,WT-With Tumor,None,None,None,NaN,None,None,None,None,NaN,NaN,released,2020-03-28T10:07:41.381880-05:00,2020-04-28T11:49:05.699379-05:00
1914,f65146ea-9d28-4dcc-9694-0520c275406d,167beb25-035b-432e-8698-c9eb9e2b6b75,0,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,NaN,WT-With Tumor,None,None,None,NaN,None,None,None,None,NaN,NaN,released,2020-03-28T10:27:07.698869-05:00,2020-04-28T11:49:05.699379-05:00
1915,b5028e73-c069-4e88-afd0-6376a7f5a666,3860f2ec-0b31-468d-a18f-1de28fb6b07e,0,377.0,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,NaN,WT-With Tumor,None,None,None,NaN,None,None,None,None,NaN,NaN,released,2020-03-28T10:40:33.443591-05:00,2020-04-28T11:49:05.699379-05:00
1916,612aa656-8e8d-44c7-a131-fc84d24f1684,3860f2ec-0b31-468d-a18f-1de28fb6b07e,0,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,NaN,WT-With Tumor,None,None,None,NaN,None,None,None,None,NaN,NaN,released,2020-03-28T10:40:33.443591-05:00,2020-04-28T11:49:05.699379-05:00


###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_clin_CGCI_followt&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**

In [0]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from case_id = ' + str(len(clin.index)))

number of case from case_id = 331


### no match 

Explanation: Row count doesn't match due to not all cases contain certain field groups. 

In [0]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'CGCI'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,8b9f18fb-9718-4012-8a61-9daf564cfba7,CGCI
1,1a6fbcfe-54d7-4785-9cf6-e596878d4101,CGCI
2,9220f346-6eab-4352-9b70-1889e3c957d8,CGCI
3,0c29ce4a-22ef-4b93-b9cc-b5a639168c2c,CGCI
4,31eee916-9abe-4cd8-a78a-423217affbc5,CGCI
...,...,...
327,8c9c36bd-ca35-4158-ae79-b317d3a61128,CGCI
328,d991215a-fd4f-434d-a17a-cf106aa72edf,CGCI
329,fcd39a3f-169a-463f-8153-17ebc1b218a8,CGCI
330,fcfdb847-5081-4086-8d32-25a281097390,CGCI


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**

In [0]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow`
group by follow__follow_up_id, case_id, follow__mol_test__count, follow__days_to_follow_up, follow__height, follow__weight, follow__bmi, follow__evidence_of_recurrence_type, follow__days_to_progression, follow__comorbidity, follow__hysterectomy_type, follow__menopause_status, follow__hormonal_contraceptive_use, follow__hiv_viral_load, follow__aids_risk_factors, follow__karnofsky_performance_status, follow__disease_response, follow__pregnancy_outcome, follow__cdc_hiv_risk_factors, follow__ecog_performance_status, follow__cd4_count, follow__progression_or_recurrence, follow__progression_or_recurrence_anatomic_site, follow__risk_factor, follow__haart_treatment_indicator, follow__days_to_recurrence, follow__nadir_cd4_count, follow__state, follow__created_datetime, follow__updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [0]:
# case_id count from the program CGCI clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI` 
group by case_id
order by count

,case_id,count
0,1a81e2c4-b637-4028-ab28-4c64bc8bab76,1
1,60acb146-43b3-44a3-9791-79284dae285e,1
2,5fadbb10-de7f-482f-8460-54d513b5de89,1
3,3adab9b3-da32-44df-9525-5f81e8cbfc02,1
4,ef5d356c-89c9-48b1-a567-519aa478f166,1
...,...,...
327,18bce2c3-5060-4282-b7a0-11ada9a3eebf,1
328,f07f2302-2276-4d5c-b52b-0b900464d39c,1
329,9d05b62c-6948-4a15-8599-503436349fbc,1
330,89c3d38f-8eda-4f62-bb9d-8d3b818c969f,1


In [0]:
# case_id count from the program CGCI_follow clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow` 
group by case_id
order by count

,case_id,count
0,a21b8ec0-1c09-48cb-97d1-42ed0096964a,2
1,3cfb178a-c476-4c0f-968e-55c57239e7c7,2
2,39ded2c0-56b5-4561-aa3f-7d417a32a131,2
3,89335258-7991-4e19-87e2-60bf747d89bd,2
4,be578313-e34c-4f26-aba6-4dc05dfd6f4d,2
...,...,...
326,008d3744-e7f0-41a5-a419-702960cf1ccb,11
327,ab45050e-2965-4d33-bff4-cd7a067ddd3d,11
328,167beb25-035b-432e-8698-c9eb9e2b6b75,11
329,3860f2ec-0b31-468d-a18f-1de28fb6b07e,11


### no match 

Explanation: Row count doesn't match due to not all cases contain certain field groups. 

##Clin CGCI_follow__mol_test

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow__mol_test`

[Table location](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_clin_CGCI_follow__mol_test&page=table)

Source : GDC API

Release version : v24

###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [0]:
#return all table information for rel24_clin_CGCI_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow__mol_test') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_CGCI_follow__mol_test,BASE TABLE


In [0]:
#return all table information for rel24_clin_CGCI_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow__mol_test') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [0]:
#check for empty schemas in dataset rel24_clin_CGCI_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow__mol_test') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below


In [0]:
#list of field descriptions for table rel24_clin_CGCI_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow__mol_test') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_CGCI_follow__mol_test

	follow__mol_test__molecular_test_id

	

rel24_clin_CGCI_follow__mol_test

	follow__follow_up_id

	Reference to ancestor follow__follow_up_id, located in rel24_clin_CGCI_follow.

rel24_clin_CGCI_follow__mol_test

	case_id

	Reference to ancestor case_id, located in rel24_clin_CGCI.

rel24_clin_CGCI_follow__mol_test

	follow__mol_test__variant_type

	The text term used to describe the type of genetic variation.

rel24_clin_CGCI_follow__mol_test

	follow__mol_test__laboratory_test

	The text term used to describe the medical testing used to diagnose, treat or further understand a patient's disease.

rel24_clin_CGCI_follow__mol_test

	follow__mol_test__specialized_molecular_test

	Text term used to describe a specific test that is not covered in the list of molecular analysis methods.

rel24_clin_CGCI_follow__mol_test

	follow__mol_test__test_result

	The text term used to describe the result of the molecular test. If the test result was a numeric value s

In [0]:
# check for empty schemas in dataset rel24_clin_CGCI_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_CGCI_follow__mol_test') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
print(clin)

                          table_name  ...                                        description
0   rel24_clin_CGCI_follow__mol_test  ...                                                   
1   rel24_clin_CGCI_follow__mol_test  ...  Reference to ancestor follow__follow_up_id, lo...
2   rel24_clin_CGCI_follow__mol_test  ...  Reference to ancestor case_id, located in rel2...
3   rel24_clin_CGCI_follow__mol_test  ...  The text term used to describe the type of gen...
4   rel24_clin_CGCI_follow__mol_test  ...  The text term used to describe the medical tes...
5   rel24_clin_CGCI_follow__mol_test  ...  Text term used to describe a specific test tha...
6   rel24_clin_CGCI_follow__mol_test  ...  The text term used to describe the result of t...
7   rel24_clin_CGCI_follow__mol_test  ...  Numeric value used to describe the upper limit...
8   rel24_clin_CGCI_follow__mol_test  ...  The text term used to describe an antigen incl...
9   rel24_clin_CGCI_follow__mol_test  ...  The text term or numeric va

###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow__mol_test`

,f0_
0,488


In [0]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_CGCI_follow__mol_test`

,f0_
0,488


In [0]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow__mol_test`

,follow__mol_test__molecular_test_id,follow__follow_up_id,case_id,follow__mol_test__variant_type,follow__mol_test__laboratory_test,follow__mol_test__specialized_molecular_test,follow__mol_test__test_result,follow__mol_test__blood_test_normal_range_upper,follow__mol_test__antigen,follow__mol_test__test_value,follow__mol_test__molecular_analysis_method,follow__mol_test__gene_symbol,follow__mol_test__state,follow__mol_test__created_datetime,follow__mol_test__updated_datetime
0,e98457e7-e1fe-40b6-855f-9d03cf4c026a,e92bc1ca-e793-46be-90bf-6890758ea48f,bcb4326c-3c7f-48e8-837a-b43bb2535a0e,None,None,None,Normal,NaN,None,NaN,FISH,MYC,released,2020-03-28T11:50:47.381246-05:00,2020-04-02T17:04:15.842910-05:00
1,3ac3fce8-503e-4b4f-bd52-19a3cb3ca220,63b48e8b-2245-437a-bc37-a19e2471db39,eb6e891a-b8ab-4261-808a-d543d1dc07d7,None,None,None,Normal,NaN,None,NaN,Not Reported,BCL2,released,2020-03-28T12:25:26.691352-05:00,2020-04-02T17:04:15.842910-05:00
2,749f513a-c942-49e1-b94b-c764a8ef1104,96c7ae9d-d8a2-4fa3-ace0-de1a0a86a74d,a7dfd3c9-4b94-4802-97c6-a096b276aeaf,None,None,None,Normal,NaN,None,NaN,FISH,MYC,released,2020-03-28T11:48:06.586052-05:00,2020-04-02T17:04:15.842910-05:00
3,945c8400-fba2-48b9-b151-cab8452423c0,e5a21e70-29df-43a7-af4a-68f57c4a96b1,d7619ad6-3a8d-40c7-9ed9-e746cfe7999c,None,None,None,Normal,NaN,None,NaN,FISH,MYC,released,2020-03-28T12:24:10.390523-05:00,2020-04-02T17:04:15.842910-05:00
4,efd2eee7-c9bf-4098-b27c-c400cd395d42,d175ae71-bbad-4e45-9d3c-771a3e9f02fb,3cfb178a-c476-4c0f-968e-55c57239e7c7,None,None,None,Negative,NaN,CD3,NaN,IHC,Not Applicable,released,2020-03-27T13:03:19.871592-05:00,2020-04-02T17:04:15.842910-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,9ec382f4-4aef-46dd-9c0a-6a62d3a74ebd,e92bc1ca-e793-46be-90bf-6890758ea48f,bcb4326c-3c7f-48e8-837a-b43bb2535a0e,None,Lactate Dehydrogenase,None,Test Value Reported,225.0,None,683.0,Other,Not Applicable,released,2020-03-28T11:50:47.381246-05:00,2020-04-02T17:04:15.842910-05:00
484,31bb7277-31fd-4cff-8613-6e7b8584cf96,c04383de-c991-4d38-b2ae-3b2d516d1d03,b59a2c3c-7085-49c5-909a-c8504b1ae2a5,None,Lactate Dehydrogenase,None,Test Value Reported,225.0,None,805.0,Other,Not Applicable,released,2020-03-27T12:34:47.887618-05:00,2020-04-02T17:04:15.842910-05:00
485,23030622-3206-4c61-8f79-ea87d44c6781,4cc1fc03-710f-444c-8732-d87fe5b04c46,ee385ea4-54db-4033-b46a-8c397ed4aea9,None,Lactate Dehydrogenase,None,Test Value Reported,225.0,None,1316.0,Other,Not Applicable,released,2020-03-28T12:28:11.829016-05:00,2020-04-02T17:04:15.842910-05:00
486,2be81296-48c9-4022-853a-90e010a25e53,d9903c50-34f1-4788-8aae-71422fd71b87,213e9f9d-7340-460c-93a3-edd559b42009,None,Lactate Dehydrogenase,None,Test Value Reported,225.0,None,861.0,Other,Not Applicable,released,2020-03-28T11:47:14.988740-05:00,2020-04-02T17:04:15.842910-05:00


###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_clin_CGCI_follow__mol_test&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**

In [0]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow__mol_test`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from submitter_id = ' + str(len(clin.index)))

number of case from submitter_id = 88


In [0]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'CGCI'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,8b9f18fb-9718-4012-8a61-9daf564cfba7,CGCI
1,1a6fbcfe-54d7-4785-9cf6-e596878d4101,CGCI
2,9220f346-6eab-4352-9b70-1889e3c957d8,CGCI
3,0c29ce4a-22ef-4b93-b9cc-b5a639168c2c,CGCI
4,31eee916-9abe-4cd8-a78a-423217affbc5,CGCI
...,...,...
327,8c9c36bd-ca35-4158-ae79-b317d3a61128,CGCI
328,d991215a-fd4f-434d-a17a-cf106aa72edf,CGCI
329,fcd39a3f-169a-463f-8153-17ebc1b218a8,CGCI
330,fcfdb847-5081-4086-8d32-25a281097390,CGCI


### no match 

Explanation: Row count doesn't match due to not all cases contain certain field groups. 

###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**

In [0]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow__mol_test`
group by follow__follow_up_id, case_id, follow__mol_test__variant_type, follow__mol_test__laboratory_test, follow__mol_test__specialized_molecular_test, follow__mol_test__test_result, follow__mol_test__blood_test_normal_range_upper, follow__mol_test__antigen, follow__mol_test__test_value, follow__mol_test__molecular_analysis_method, follow__mol_test__gene_symbol, follow__mol_test__state, follow__mol_test__created_datetime, follow__mol_test__updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [0]:
# case_id count from the program CGCI clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI` 
group by case_id
order by count

,case_id,count
0,1a81e2c4-b637-4028-ab28-4c64bc8bab76,1
1,60acb146-43b3-44a3-9791-79284dae285e,1
2,5fadbb10-de7f-482f-8460-54d513b5de89,1
3,3adab9b3-da32-44df-9525-5f81e8cbfc02,1
4,ef5d356c-89c9-48b1-a567-519aa478f166,1
...,...,...
327,18bce2c3-5060-4282-b7a0-11ada9a3eebf,1
328,f07f2302-2276-4d5c-b52b-0b900464d39c,1
329,9d05b62c-6948-4a15-8599-503436349fbc,1
330,89c3d38f-8eda-4f62-bb9d-8d3b818c969f,1


In [0]:
# case_id count from the program CGCI_follow__mol_test clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow__mol_test` 
group by case_id
order by count

,case_id,count
0,886e1609-e89c-4d5e-ab9d-4f9ae18c937a,1
1,c334fa41-78ea-41ab-97e7-44db7fa8b460,1
2,b06f7bf0-45bb-4aca-8f22-72d8a4f73f99,1
3,7277e644-f3da-414a-9821-8037cc99ef9f,1
4,f4a0bef2-c397-4149-830f-8d5f62e1b5e8,1
...,...,...
83,fcd39a3f-169a-463f-8153-17ebc1b218a8,9
84,d991215a-fd4f-434d-a17a-cf106aa72edf,9
85,eb6e891a-b8ab-4261-808a-d543d1dc07d7,10
86,2d9d88c8-d827-4bc8-88c6-75e0d0c3f988,10


### no match 

Explanation: Row count doesn't match due to not all cases contain certain field groups. 

###test 7 - follow__follow_up_id count verification

**7. QC follow__follow_up_id count from parent diag table if applicable**

In [0]:
# follow__follow_up_id count from the program CGCI_follow clinical table

%%bigquery --project isb-project-zero

select distinct follow__follow_up_id, count(follow__follow_up_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow` 
group by follow__follow_up_id
order by count

,follow__follow_up_id,count
0,27640af3-cdfb-4904-9341-d65f1b47517e,1
1,d175ae71-bbad-4e45-9d3c-771a3e9f02fb,1
2,085e05df-f73c-46fb-a297-43f245e40491,1
3,53de1b44-46b8-4656-90b2-ad41a6ff8e24,1
4,5b5fd590-d03c-481a-a7d4-3faed33bc3f1,1
...,...,...
1913,9a24df76-6ce4-40a5-9c2f-0ab5da97930f,1
1914,f65146ea-9d28-4dcc-9694-0520c275406d,1
1915,b5028e73-c069-4e88-afd0-6376a7f5a666,1
1916,612aa656-8e8d-44c7-a131-fc84d24f1684,1


In [0]:
# follow__follow_up_id count from the program CGCI_follow__mol_test clinical table

%%bigquery --project isb-project-zero

select distinct follow__follow_up_id, count(follow__follow_up_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_CGCI_follow__mol_test` 
group by follow__follow_up_id
order by count

,follow__follow_up_id,count
0,9af46e2b-0463-4562-a1fc-dcb273d3b531,1
1,29ac316f-8147-4cda-b724-d37f919817cf,1
2,ff29cf34-de9c-438f-8415-715a42283130,1
3,44bdb77d-81bd-4582-be77-1c1ffe53c129,1
4,f5d0f77a-1937-435f-b994-ee059b7a6057,1
...,...,...
83,552538aa-2b4d-4ca8-b286-baadea98c70d,9
84,f79e9c80-6fd3-4155-b461-8a6a932d281a,9
85,63b48e8b-2245-437a-bc37-a19e2471db39,10
86,2c4e6d3b-7af5-4dd1-928f-65cfdc8875f2,10


### no match 

Explanation: Row count doesn't match due to not all cases contain certain field groups. 